## Stable Diffusion

In [ ]:
! python -m pip install --upgrade pip
! pip install botocore --upgrade --quiet

In [ ]:
!pip list|grep -i diffuser

In [ ]:
!echo "Y"|pip uninstall diffusers                     

In [ ]:
!pip install -r /home/ec2-user/SageMaker/sm_sample/deepspeed/requirements.txt

In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
import torch
print(torch.__version__)  # 查看torch当前版本号
print(torch.version.cuda)  # 编译当前版本的torch使用的cuda版本号
print(torch.cuda.is_available())  # 查看当前cuda是否可用于当前版本的Torch，如果输出

In [ ]:
import os
import json
import boto3
import sagemaker
import uuid
import torch
from torch import autocast
from PIL import Image
import io
import requests
import traceback
import os
import json
import torch
from diffusers import StableDiffusionPipeline
from diffusers import StableDiffusionImg2ImgPipeline
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import deepspeed



#model_dir='/root/dreambooth/models_diffuser/'
#model_dir='/home/ec2-user/SageMaker/dst/'
model_dir='andite/anything-v4.0'
#model_dir='/home/ec2-user/SageMaker/dst/'
pipeline1 = StableDiffusionPipeline.from_pretrained(model_dir, torch_dtype=torch.float16)
pipeline2 = StableDiffusionPipeline.from_pretrained(model_dir, torch_dtype=torch.float16)
print("model loaded:",pipeline1)
 
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
pipeline1.to("cuda")
pipeline1.enable_attention_slicing()  
pipeline2.to("cuda")
pipeline2.enable_attention_slicing()  

try:
   print("begin load deepspeed....")    
   deepspeed.init_distributed()
   engine = deepspeed.init_inference(
       model=getattr(pipeline2,"model", pipeline2),      # Transformers models
       mp_size=1,        # Number of GPU
       dtype=torch.float16, # dtype of the weights (fp16)
       replace_method="auto", # Lets DS autmatically identify the layer to replace
       replace_with_kernel_inject=True, # replace the model with the kernel injector
   )
#
   if hasattr(pipeline2, "model"):
       pipeline4.model = engine

   print('model accelarate with deepspeed!')
except Exception as e:
   print("deepspeed accelarate excpetion!")
   print(e)
    


### Inference

In [ ]:
from PIL import Image
from io import BytesIO
from IPython.display import display
import base64
import matplotlib.pyplot as plt

# helper decoder
def decode_base64_image(image_string):
  base64_image = base64.b64decode(image_string)
  buffer = BytesIO(base64_image)
  return Image.open(buffer)

# display PIL images as grid
def display_images(images=None,columns=3, width=100, height=100):
    plt.figure(figsize=(width, height))
    for i, image in enumerate(images):
        plt.subplot(int(len(images) / columns + 1), columns, i + 1)
        plt.axis('off')
        plt.imshow(image)
        
def get_bucket_and_key(s3uri):
    pos = s3uri.find('/', 5)
    bucket = s3uri[5 : pos]
    key = s3uri[pos + 1 : ]
    return bucket, key        

In [ ]:
import time
s3_client = boto3.client('s3')
prompt = "A dog trying catch a flying pizza art drawn by disney concept artists, golden colour, high quality, highly detailed, elegant, sharp focus"
#prompt = "callie, inkling, splatoon, cute, forest background"
#negative_prompt='(worst quality, low quality:1.4), simple background'
#seed=270522672
seed=546576788905676

#prompt = "priest, blue robes, 68 year old nun, national geographic, portrait, photo, photography"
#prompt = "hotel room with a swimming pool outside of the window, TV on the table, moon in the sky"

start=time.time()
generator = torch.Generator(device='cuda').manual_seed(seed)
images = pipeline2(prompt, num_images_per_prompt=2, num_inference_steps=30, guidance_scale=7.5,generator=generator).images
print(f"Time taken: {time.time() - start}s")
ret_images=[]
for image in images:
   bucket, key = get_bucket_and_key("s3://sagemaker-us-west-2-687912291502/deepspeed/")
   key = '{0}{1}.png'.format(key, uuid.uuid4())
   buf = io.BytesIO()
   image.save(buf, format='PNG')
   s3_client.put_object(
       Body = buf.getvalue(), 
       Bucket = bucket, 
       Key = key, 
       ContentType = 'image/PNG'
   )
   ret_images.append(image)
   print('image: ', 's3://{0}/{1}'.format(bucket, key))



In [ ]:
display_images(ret_images)

In [ ]:
# visualize generation
display_images(ret_images)